## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint
7. After this lab is completed, delete the endpoint to avoid unnecessary charges

In [1]:
# Install SageMaker 2.x version.
#!pip install --upgrade sagemaker

In [1]:
import boto3
import sagemaker
import math
import dateutil
import re
import numpy as np

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [2]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges

boto_session = boto3.Session(profile_name='ml_user_predict', region_name='us-east-1')

#boto_session = boto3.Session()

In [3]:
type(boto_session)

boto3.session.Session

In [4]:
sess = sagemaker.Session(boto_session=boto_session)

In [5]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
#endpoint_name = 'xgboost-biketrain-v1'
endpoint_name = 'linear-learner-2022-11-15-14-57-02-049'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [6]:
predictor

In [7]:
# We are sending data for inference in CSV format
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()
#predictor.deserializer = json_deserializer

In [8]:
X_test = np.array([[ 4.5],
                   [ 3. ],
                   [11.5],
                   [ 2.9],
                   [ 9.5],
                   [ 5.3],
                   [ 9. ]])

In [9]:
X_test

array([[ 4.5],
       [ 3. ],
       [11.5],
       [ 2.9],
       [ 9.5],
       [ 5.3],
       [ 9. ]])

In [10]:
# Let's invoke prediction now
#predictor.predict(transform_data(sample_one))

result = predictor.predict(X_test)
result

{'predictions': [{'score': 68126.765625},
  {'score': 53036.640625},
  {'score': 138547.375},
  {'score': 52030.6328125},
  {'score': 118427.1953125},
  {'score': 76174.84375},
  {'score': 113397.15625}]}

In [11]:
# Since the result is in json format, we access the scores by iterating through the scores in the predictions

predictions = np.array([r['score'] for r in result['predictions']])

In [12]:
predictions

array([ 68126.765625 ,  53036.640625 , 138547.375    ,  52030.6328125,
       118427.1953125,  76174.84375  , 113397.15625  ])

In [13]:
# Don't forget to delete the endpoint
# From SageMaker Console, Select "Endpoints" under Inference and Delete the Endpoint